In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
from tableone import TableOne
from scipy import stats
import random
from deap import creator
from deap import base
from deap import tools
from deap import algorithms
import matplotlib.pyplot as plt
import re
from sklearn import linear_model
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

In [77]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv"

In [78]:
df = pd.read_csv(url)
df

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,56037,43051,6104,14.178532,6326,14.694200,5359,12.448027,6577,15.277229,...,2098,8.9,8.3,9.6,2834,2.6,2.4,2.8,821,5
3136,56039,23081,2384,10.328842,2185,9.466661,2967,12.854729,4093,17.733200,...,928,7.2,6.5,8.0,1360,2.4,2.2,2.6,447,5
3137,56041,20299,3121,15.375142,3205,15.788955,2153,10.606434,2702,13.311001,...,1163,10.4,9.5,11.2,1500,3.0,2.8,3.2,430,5
3138,56043,7885,858,10.881420,1113,14.115409,715,9.067850,903,11.452124,...,506,11.3,10.3,12.1,686,3.4,3.2,3.7,207,6


In [79]:
df = df.drop_duplicates()
df= df.reset_index(drop = True)

In [80]:
def is_binary(df_, nums):
    df = df_.copy()
    variables = []
    for var in nums:
        flag = True
        unique = df_[var].unique()
        for value in unique:
            if value not in [0, 1, np.nan, 0.0, 1.0]:
                flag = False
        if flag == True:
            variables.append(var)
    return variables


def breakdown_vars(df):
    """
    This function allow us categorize accodign to numerical or not
    """
    categorial = []
    nonormal = []
    normal = []
    binaries = is_binary(df, df.columns)
    for t in df.columns:
            if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category') and t not in binaries:
                categorial.append(t)
            if (df[t].dtypes=="int64" or df[t].dtypes=="float64") and t not in binaries:
              n,p = stats.shapiro(df[t])
              if p<0.05:
                nonormal.append(t)
              else:
                normal.append(t)
    return categorial, binaries, nonormal, normal

def score(solution, classifier, X_train_, y_train_, metric):
    """
    Classifier must have the fit and predict methods (sklearn)
    """
    X_train, y_train = X_train_.copy(), y_train_.copy()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.30, shuffle = True,
                                                    random_state = 666)
    solution = np.array(solution, dtype='bool')
    X_train = X_train.loc[:,solution]
    X_val = X_val.loc[:,solution]
    return metric(y_val, classifier(X_train, y_train).predict(X_val)),


def grid_lasso(X_train, y_train):
    model = linear_model.Lasso(random_state=1000)
    alpha = np.array([0.01, 0.1, 1])
    grid = dict(alpha = alpha)
    cv = KFold(n_splits=5, shuffle=True, random_state=10)
    grid_search = GridSearchCV(estimator=model,
                               param_grid=grid,
                               cv=cv,
                               scoring=mean_squared_error)
    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_


In [81]:
df = pd.read_csv(url)
target = "Obesity_number"

In [82]:
X = df.drop(columns=target).copy()
y = df[target].copy()

In [83]:
to_drop = [var for var in X.columns if len(re.findall(r'Heart*', var, re. IGNORECASE))>0]
print(to_drop)
X.drop(columns=to_drop, inplace=True)

['Heart disease_prevalence', 'Heart disease_Lower 95% CI', 'Heart disease_Upper 95% CI', 'Heart disease_number']


In [84]:
categorial, binaries, nonormal, normal = breakdown_vars(df)

In [85]:
X.drop(columns = categorial, inplace=True)

In [86]:
for var in X.columns:
  if var in nonormal + normal:
    corr = np.corrcoef(y, df[var])[0][1]
    if abs(corr) > 0.99:
      print(corr, var)

0.9903977737125736 TOT_POP
0.9914744170496121 0-9
0.9908361018660831 19-Oct
0.9903977737125736 POP_ESTIMATE_2018
0.9985277320873697 anycondition_number


In [87]:
to_drop = [var for var in X.columns if len(re.findall(r'Total pop*', var, re. IGNORECASE))>0]
X.drop(columns= to_drop[:-1], inplace=True)

In [88]:
to_drop = [var for var in X.columns if len(re.findall(r'^[0-9]+', var, re. IGNORECASE))>0]
X.drop(columns= to_drop, inplace=True)

In [89]:
to_drop = [var for var in X.columns if len(re.findall(r'95%', var, re. IGNORECASE))>0]
X.drop(columns= to_drop, inplace=True)

In [90]:
to_drop = [var for var in X.columns if len(re.findall(r'prevalence', var, re. IGNORECASE))>0]
X.drop(columns= to_drop, inplace=True)

In [91]:
to_drop = [var for var in X.columns if len(re.findall(r'\%', var, re. IGNORECASE))>0]
X.drop(columns= to_drop, inplace=True)

In [92]:
to_drop = [var for var in X.columns if len(re.findall(r'pop\b', var))>0]
X.drop(columns= to_drop, inplace=True)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,
                                                    test_size=0.20,
                                                    random_state = 42)

In [94]:
best_model = grid_lasso(X_train, y_train) #entreno el modelo con X_train y y_train
preds = best_model.predict(X_test)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.902e+09, tolerance: 1.322e+09
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 191, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "/usr/local/lib/python3.10/inspect.py", line 318

In [95]:
print(mean_squared_error(y_test, preds))
print(r2_score(y_test, preds))

11696523.000949241
0.9946507891556711


In [96]:
X_train[target] = list(y_train)
X_test[target] = list(y_test)
X_train.to_csv("/workspaces/Regularized-Linear-Regression-Project/data/processed/clean_train.csv", index = False)
X_test.to_csv("/workspaces/Regularized-Linear-Regression-Project/data/processed/clean_test.csv", index = False)